In [1]:
import pandas as pd
import requests
import json

In [3]:
# Key file not included in repo
with open("./API_key") as fin:
    key = fin.read().strip()

# Get the stock symbols for `S&P 500`

In [27]:
resp = json.loads(
    requests.get(f"https://eodhistoricaldata.com/api/fundamentals/GSPC.INDX?api_token={key}").text
)

In [28]:
symbols = [c["Code"] for c in resp["Components"].values()]
print(f"Got {len(symbols)} stocks in total\nExample: {symbols[:5]}")

Got 503 stocks in total
Example: ['AIZ', 'MNST', 'MTCH', 'PGR', 'FISV']


# Get fundamentals data

In [6]:
import tqdm.notebook

In [86]:
fundamentals = {}

In [88]:
for sym in tqdm.notebook.tqdm(symbols):
    if sym in fundamentals:
        continue
        
    resp = json.loads(
        requests.get(f"https://eodhistoricaldata.com/api/fundamentals/{sym}?api_token={key}").text
    )
    
    fundamentals[sym] = resp

  0%|          | 0/503 [00:00<?, ?it/s]

In [101]:
%%time
with open("./fundamentals_SP500.json", "w") as fout:
    json.dump(fundamentals, fout)

CPU times: total: 766 ms
Wall time: 2min 8s


In [2]:
%%time
with open("./fundamentals_SP500.json", "r") as fin:
    fundamentals = json.load(fin)

CPU times: total: 2.33 s
Wall time: 3.32 s


# Convert JSON to DataFrame

In [3]:
js = fundamentals["A"]

In [8]:
js['Financials']["Balance_Sheet"]["quarterly"]

{'2022-07-31': {'date': '2022-07-31',
  'filing_date': '2022-09-01',
  'currency_symbol': 'USD',
  'totalAssets': '10484000000.00',
  'intangibleAssets': '849000000.00',
  'earningAssets': None,
  'otherCurrentAssets': '258000000.00',
  'totalLiab': '5393000000.00',
  'totalStockholderEquity': '5091000000.00',
  'deferredLongTermLiab': None,
  'otherCurrentLiab': '666000000.00',
  'commonStock': '3000000.00',
  'capitalStock': '3000000.00',
  'retainedEarnings': '139000000.00',
  'otherLiab': '759000000.00',
  'goodWill': '3948000000.00',
  'otherAssets': '749000000.00',
  'cash': '1071000000.00',
  'cashAndEquivalents': '1071000000.00',
  'totalCurrentLiabilities': '1902000000.00',
  'currentDeferredRevenue': '498000000.00',
  'netDebt': '1841000000.00',
  'shortTermDebt': '180000000.00',
  'shortLongTermDebt': '180000000.00',
  'shortLongTermDebtTotal': '2912000000.00',
  'otherStockholderEquity': '5311000000.00',
  'propertyPlantEquipment': '1054000000.00',
  'totalCurrentAssets': '

In [9]:
pd.json_normalize(
        js['Financials']["Balance_Sheet"]["quarterly"].values(),   
)

,date,filing_date,currency_symbol,totalAssets,intangibleAssets,earningAssets,otherCurrentAssets,totalLiab,totalStockholderEquity,deferredLongTermLiab,...,preferredStockRedeemable,capitalSurpluse,liabilitiesAndStockholdersEquity,cashAndShortTermInvestments,propertyPlantAndEquipmentGross,propertyPlantAndEquipmentNet,accumulatedDepreciation,netWorkingCapital,netInvestedCapital,commonStockSharesOutstanding
0,2022-07-31,2022-09-01,USD,10484000000.00,849000000.00,None,258000000.00,5393000000.00,5091000000.00,None,...,None,5311000000.00,10484000000.00,1077000000.00,None,1054000000.00,None,1788000000.00,8003000000.00,299000000.00
1,2022-04-30,2022-05-31,USD,10455000000.00,898000000.00,None,262000000.00,5333000000.00,5122000000.00,None,...,None,5292000000.00,10455000000.00,1207000000.00,None,1010000000.00,None,1830000000.00,8027000000.00,301000000.00
2,2022-01-31,2022-03-03,USD,10327000000.00,929000000.00,None,232000000.00,5173000000.00,5154000000.00,None,...,None,5290000000.00,10327000000.00,1158000000.00,None,974000000.00,None,1890000000.00,7884000000.00,303000000.00
3,2021-10-31,2021-12-17,USD,10705000000.00,981000000.00,None,222000000.00,5316000000.00,5389000000.00,None,...,None,5320000000.00,10705000000.00,1575000000.00,None,945000000.00,None,2091000000.00,8118000000.00,305000000.00
4,2021-07-31,2021-09-01,USD,10491000000.00,1032000000.00,None,264000000.00,5545000000.00,4946000000.00,None,...,None,5307000000.00,10491000000.00,1428000000.00,905000000.00,905000000.00,None,1908000000.00,7804000000.00,306000000.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90,2000-01-31,2000-03-15,USD,7107000000.00,None,None,560000000.00,2621000000.00,4486000000.00,None,...,None,None,7107000000.00,1410000000.00,1408000000.00,1408000000.00,None,None,None,440000000.00
91,1999-10-31,1999-10-31,USD,5444000000.00,None,None,404000000.00,2062000000.00,3382000000.00,None,...,None,None,5444000000.00,None,1387000000.00,1387000000.00,None,None,None,440000000.00
92,1999-07-31,1999-07-31,USD,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,450000000.00
93,1999-04-30,1999-04-30,USD,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,461764706.00


# Get monthly stock data

In [13]:
import io

In [15]:
historical = {}

In [16]:
for sym in tqdm.notebook.tqdm(symbols):
    if sym in historical:
        continue
    
    # Monthly historical price
    df = pd.read_csv(
        io.StringIO(
            requests.get(f"https://eodhistoricaldata.com/api/eod/{sym}?api_token={key}&period=m").text)
    )
    df["symbol"] = sym
    
    historical[sym] = df

  0%|          | 0/503 [00:00<?, ?it/s]

In [19]:
df_hist = pd.concat(historical.values())

In [24]:
df_hist.to_csv("./monthly_price.csv", index=False)